In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad

from scroutines import powerplots

In [2]:
outdir = "/data/qlyu/v1/results_merfish/plots_230621"
outdatadir = "/data/qlyu/v1/results_merfish"
!mkdir -p $outdir

In [3]:
f1 = '/data/qlyu/v1/data/merfish/merfish_06142023/pos/region0/cell_by_gene.csv'
f2 = '/data/qlyu/v1/data/merfish/merfish_06142023/pos/region0/cell_metadata.csv'
output = outdatadir + '/pos_reg0_230719.h5ad'

df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)

df1.shape, df2.shape

((466790, 551), (466790, 17))

In [4]:
genes = df1.iloc[:,1:-50].columns
blnks = df1.iloc[:, -50:].columns
df = df2.join(df1)
assert np.all(df['cell'] == df['EntityID'])
len(genes), len(blnks)

(500, 50)

In [5]:
# sns.histplot(df2['volume'], cumulative=True, element='step')

In [6]:
# sns.histplot(df1[blnks].sum(axis=1), cumulative=True, element='step')

<Axes: xlabel='gncov', ylabel='Count'>

In [7]:
df['fpcov'] = df[blnks].sum(axis=1)
df['gncov'] = df[genes].sum(axis=1)
sns.histplot(df['gncov'], cumulative=True, element='step')

In [8]:
print(df.shape)
cond = np.all([
    df['volume'] < 5000, 
    df['fpcov'] < 5,
    df['gncov'] > 10,
], axis=0)

df = df[cond] 
print(df.shape)

(466790, 570)
(291335, 570)


In [9]:
# functions (the teacher wrote for you to use later)
def rot2d(x, y, theta, unit='degree'):
    """ rotate data points defined by `x` and `y` by `theta` degree
    """
    a = np.vstack([x,y]).T
    if unit == 'degree':
        theta = theta*np.pi/180 # convert to radian

    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    ar = a.dot(R.T)
    return ar[:,0], ar[:,1]

def st_scatter(x, y, gexp=None, vmax_p=98, title='', s=1, cbar_label='', output='', cmap='rocket_r', axis_off=True):
    """customized scatter plot -- yesterday's progress
    """
  
    fig, ax = plt.subplots(figsize=(10,8))
    if gexp is not None:
        vmax = np.percentile(gexp, vmax_p)
        g = ax.scatter(x, y, c=gexp, s=s, edgecolor='none', vmax=vmax, cmap=cmap, rasterized=True)
        fig.colorbar(g, label=cbar_label, shrink=0.3)
    else:
        g = ax.scatter(x, y, s=s, edgecolor='none', cmap=cmap, rasterized=True)
  
    if axis_off:
        ax.axis('off')
    ax.set_title(title)
    ax.set_aspect('equal')
  
    if output:
        powerplots.savefig_autodate(fig, output)
        
    return 

In [10]:
# calibrate coordinates

x = df['center_x']
y = df['center_y']
xr, yr = rot2d(x, y, 90)
df['x'] = xr
df['y'] = yr

gn = 'Slc17a7'
g = np.log10(1+df[gn])
st_scatter(xr, yr, gexp=g, title=gn, cbar_label='log10(cnt+1)', axis_off=False)

In [11]:
# focus on Visual cortex
cond_region = df['y'] > 10000
dfsub = df[cond_region]

In [12]:
gn = 'Slc17a7'
g = np.log10(1+dfsub[gn])
xr = dfsub['x']
yr = dfsub['y']
st_scatter(xr, yr, gexp=g, title=gn, cbar_label='log10(cnt+1)', axis_off=False)

In [13]:
dfsub

,EntityID,fov,volume,center_x,center_y,min_x,min_y,max_x,max_y,anisotropy,...,Blank-45,Blank-46,Blank-47,Blank-48,Blank-49,Blank-50,fpcov,gncov,x,y
842,2596588900021100014,1267,47.282381,10030.044005,-106.474372,10025.949138,-110.061305,10034.695592,-101.855788,1.420779,...,0,0,0,0,0,0,0,13,106.474372,10030.044005
843,2596588900021100015,1267,45.469704,10007.599697,-97.044473,10003.841445,-101.306778,10010.743873,-93.659784,1.632384,...,0,0,0,0,0,0,0,26,97.044473,10007.599697
848,2596588900021100021,1267,215.311913,10000.653732,-87.957751,9995.148721,-92.018865,10005.879309,-83.932158,1.445268,...,0,0,0,0,0,0,1,86,87.957751,10000.653732
850,2596588900021100025,1267,492.809448,10022.756877,-78.119639,10013.507671,-83.491605,10030.805391,-72.700264,1.599436,...,0,0,0,0,0,0,4,175,78.119639,10022.756877
852,2596588900021100027,1267,56.909914,10013.809800,-73.420498,10008.767707,-77.636585,10017.740660,-69.360011,1.305967,...,0,0,0,0,0,0,0,20,73.420498,10013.809800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466733,2596588904130100012,4,50.363779,10056.193847,10121.691912,10052.541000,10117.893419,10059.234919,10125.266708,1.107079,...,0,0,0,0,0,0,0,17,-10121.691912,10056.193847
466736,2596588904130100017,4,745.478771,10164.490921,10116.673266,10154.490994,10107.508917,10174.144659,10126.620840,1.051687,...,0,0,0,0,0,0,0,28,-10116.673266,10164.490921
466739,2596588904130100020,4,147.861053,10046.752690,10109.995860,10040.732775,10106.684786,10052.850668,10113.324588,1.818748,...,0,0,0,0,0,0,0,24,-10109.995860,10046.752690
466746,2596588904131100003,3,17.057492,10209.186796,10111.763393,10207.045335,10109.818167,10212.028839,10113.540586,1.489275,...,0,0,0,0,0,0,0,11,-10111.763393,10209.186796


In [14]:
df2.columns

Index(['EntityID', 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y',
       'max_x', 'max_y', 'anisotropy', 'transcript_count',
       'perimeter_area_ratio', 'solidity', 'DAPI_raw', 'DAPI_high_pass',
       'PolyT_raw', 'PolyT_high_pass'],
      dtype='object')

In [15]:
metacols = np.hstack(['x', 'y', 'gncov', 'fpcov', df2.columns])
adata = ad.AnnData(X=dfsub[genes].values, 
                   obs=dfsub[metacols], 
                   var=pd.DataFrame(index=genes), 
                   obsm=dict(blanks=dfsub[blnks].values),
                  )

/tmp/ipykernel_1716617/4156774178.py:2: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=dfsub[genes].values,
/home/qlyu/mambaforge/envs/bbbroutine/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [16]:
adata.write(output)

In [17]:
!ls $outdatadir/*230719.h5ad

/data/qlyu/v1/results_merfish/ant_reg0_230719.h5ad
/data/qlyu/v1/results_merfish/ant_reg1_230719.h5ad
/data/qlyu/v1/results_merfish/pos_reg0_230719.h5ad
/data/qlyu/v1/results_merfish/pos_reg1_230719.h5ad


In [18]:
# for gn in genes:
#     g = np.log10(1+df[gn])
#     output = outdir + f'/{gn}.pdf'
#     print(output)
#     st_scatter(xr, yr, gexp=g, title=gn, cbar_label='log10(cnt+1)', output=output)
#     plt.close()
#     # break